# Reinforcement Learning

In this lab, you will have the opportunity to explore Reinforcement Learning (RL) algorithms in practice, training agents to navigate through a 2D grid world to maximize their rewards. In a sequence of tasks, you will 

1. discover the effect of randomness on RL training,
2. compare different RL algorithms trained on the same task, 
3. implement the Q-Learning algorithm
4. explore whether the task is harder to learn if the agent started in a different random position in each episode, and
5. explore different tasks by modifying the environment's reward structure

## Setup

We start by importing dependencies and setting up some functions that will be useful in completing these tasks.

### Imports

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from grid_world import GridWorld # ensure the file grid_world.py is located in the same folder as this notebook.

### Environment

The class below defines the grid world environment that we've already seen in the lecture. It consists of a grid with 4 rows and 5 columns, specifying 20 environment states (one state for each grid cell). In each episode, the agent starts in state (0, 3). The cell (4, 0) is declared a terminal state, ending an episode whenever the agent transitions into that state. All state transitions yield a reward of 0 by default. The reward for transitions into two specific states are set differently, yielding a reward of 1 when an agent's action results in a transition into state (4, 0), and yielding a reward of -1 when an agent's action results in a transition into state (1, 2).

In [ ]:
class ExampleGrid(GridWorld):
    
    def __init__(self):
        super().__init__(rows=4, cols=5, agent_start=(0, 3))
        #self.rewards = -np.ones_like(self.rewards)
        self.update_terminal(4, 0) # make state (4, 0) a terminal state
        self.update_reward(4, 0, 1) # make transitions into state (4, 0) yield a reward of 1
        self.update_reward(1, 2, -1) # make transitions into state (1, 2) yield a reward of -1
        
env = ExampleGrid() # instantiate the example grid
env.render() # display the environment state(green dot), terminal states (gray shaded cells), and rewards (numbers inside cells)

### Tabular Softmax Agent

This agent class implements an explicit policy where the probability distribution over actions given a state $\pi(a|s)$ is defined as the softmax over action-specific 'logits' $z$:
\begin{equation}
\pi(a=a_i|s) = \sigma(z_{s,a}) = \frac{e^{z_{s, a_i}}}{\sum_j e^{z_{s, a_j}}}
\end{equation}

Using a tabular state representation, the policy is parameterized with a separate parameter for each state-action pair $z_{s, a}$ with state $s: (x, y) \in \mathcal{S}$ and action $a \in \mathcal{A}$.

In [ ]:
class SoftmaxAgent():
    """
    A tabular agent parameterizing the logits of a softmax policy.
    
    Args:
        z (np.array): Parameters organized by [y, x, a], where (y, x) encode
        the state (as a cell in a grid world) and 'a' each possible action.
    """
    def __init__(self, z):
        self.z = z
        
    def act(self, state, epsilon):
        """ sample from the policy pi(a, s), where pi is defined as the 
        softmax function with parameters self.z[y, x]."""
        x, y = state
        p = np.exp(self.z[y, x])/np.sum(np.exp(self.z[y, x]))
        a = np.random.choice(len(p), p=p)
        return a
    
    def rollout(self, env, epsilon=0):
        """ complete a full episode interacting with the environment 
        by following the current policy."""
        
        states, actions, rewards = [], [], []
        done = False

        s = env.reset()
        states.append(s)
        while not done:
            a = self.act(s, epsilon)
            s, r, done, _ = env.step(a)
            states.append(s)
            actions.append(a)
            rewards.append(r)
        
        return states, actions, rewards
    
    def render_pi(self, env, ax=None):
        """ Display the learned policy. """
        if ax is None:
            fig, ax = env._create_fig()
        env.render_grid(ax)
        pi = np.exp(self.z) / np.expand_dims(np.sum(np.exp(self.z), axis=2), axis=2)
        for y in range(env.rows):
            for x in range(env.cols):
                # render distribution over actions
                p =  pi[y, x]
                p_max = np.max(p)
                for a in range(len(env.action_space)):
                    length = p[a]/(1e-6+p_max)*0.3
                    env.render_action(ax, action=env.action_space[a], agent=(x, y), length=length, hw=0.1, hl=0.15, ox=0, oy=0)

## Tasks

### 1. How does randomness affect training of REINFORCE ?

The code snippet below trains a Tabular Softmax policy with REINFORCE for 1000 rollouts (episodes) and then plots the return of each rollout. It also illustrates the trained policy and shows an example rollout from the last training step.

Write a function that repeats training 5 times and then plots mean, min and max reward after each rollout.

In [ ]:
# instantiate environment
env = ExampleGrid()
# instantiate policy
logits = np.ones((env.rows, env.cols, len(env.action_space)))
agent = SoftmaxAgent(z=logits)
# define learning rate
eta = 0.01
gamma = 0.95

# train for 300 episodes and record episode returns
returns = []
for i in range(1000):
    
    states, actions, rewards = agent.rollout(env)
    G = np.sum(rewards* gamma**np.arange(len(rewards))) # here, the decay gamma is omitted, implicitly assuming it to be 1.
    returns.append(G)
    # question for students: why do we subtract the mean of past returns for training?
    G -= np.mean(returns)
    z = agent.z
    dz = np.zeros_like(z)
    for t in range(len(states)-1):
        
        x, y = states[t]
        a = actions[t]
        # estimate dz as (delta_a - 1)
        dz[y,x] -= np.ones(4)
        dz[y,x, a] += 1

    # scale the gradient by the episode return
    dz = dz * G
    # apply gradient with learning rate eta
    z = agent.z + eta * dz
    # saturate logits to avoid distribution collapse
    agent.z = np.maximum(-6, np.minimum(6, z))

#agent.render_pi(env)
#plt.title(f'REINFORCE policy after rollout {i+1}')
returns_reinforce = returns

fig, ax = plt.subplots(3, 1, figsize=(10, 24))
ax[0].plot(returns_reinforce)
ax[0].set_title('REINFORCE training return of each rollout')
ax[0].set_ylabel('Return')
ax[0].set_xlabel('Rollout')
agent.render_pi(env, ax[1])
ax[1].set_title('REINFORCE learned policy after training.')
env.render_rollout(states, ax=ax[2])
ax[2].set_title('REINFORCE example rollout after training.')

## 2. How does Actor-Critic compare to REINFORCE in training performance?

The code snippet below trains a Tabular Softmax policy with Actor-Critic for 1000 rollouts (episodes) and then plots the return of each rollout. It also illustrates the trained policy and shows an example rollout from the last training step.

Train Actor-Critic 5 times and compare mean, min and max return in each rollout with the mean min and max return observed during REINFORCE training.

In [ ]:
# Actor-Critic
# instantiate environment
env = ExampleGrid()
# instantiate policy
logits = np.ones((env.rows, env.cols, len(env.action_space)))
agent = SoftmaxAgent(z=logits)
# define learning rate
eta = 0.2
gamma = 1. # discount factor

# initialize the critic
Q = np.zeros_like(logits)

returns = []
for i in range(1000):
    
    states, actions, rewards = agent.rollout(env)
    G = np.sum(rewards)
    returns.append(G)
    # question for students: why do we subtract the mean of past returns?
    G -= np.mean(returns)
    p = np.exp(agent.z) / np.expand_dims(np.sum(np.exp(agent.z), axis=2), axis=2)
    dz = np.zeros_like(agent.z)
    for t in range(len(states)-1):
        
        x, y = states[t]
        a = actions[t]
        
        # estimate updated Q
        x_next, y_next = states[t+1]
        v = np.sum(Q * p, axis=2)[y_next, x_next]
        Q[y, x, a] = rewards[t] + gamma * v
        
        # estimate gradient
        # here, we scale gradients with the relevant Q-value in each step 
        # instead of using the episode return at the end.
        dz[y, x] -= Q[y, x, a]
        dz[y, x, a] += Q[y, x, a]
    
    # update parameters (question for students: what does the 0.99 accomplish?)
    z = 0.99*agent.z + eta * dz
    # saturate parameters to prevent distribution collapse
    agent.z = np.maximum(-6, np.minimum(6, z))

returns_ac = returns

fig, ax = plt.subplots(3, 1, figsize=(10, 24))
ax[0].plot(returns_ac)
ax[0].set_title('Actor-Critic training return of each rollout')
ax[0].set_ylabel('Return')
ax[0].set_xlabel('Rollout')
agent.render_pi(env, ax[1])
ax[1].set_title('Actor-CriticNFORCE learned policy after training.')
env.render_rollout(states, ax=ax[2])
ax[2].set_title('Actor-Critic example rollout after training.')

## 3. Implement the Q-Learning algorithm


### 3.1 Action Selection

Implement the ```act``` method of the `QAgent` below to select the action with maximum Q-value. 

Amend your implementation to an $\epsilon$-greedy policy that returns a random action with probability $\epsilon$ and returns the action with maximum Q-value with probability $(1-\epsilon)$.

In [ ]:
class QAgent(SoftmaxAgent):
    """
    A tabular agent parameterizing the Q-values corresponding to the policy.
    
    Args:
        q (np.array): Parameters organized by [y, x, a], where (y, x) encode
                    the state (as a cell in a grid world) and 'a' each possible action.
        
    """
    def __init__(self, q):
        self.q = q
        
    def act(self, state, epsilon=0.1):
        """ select the action with maximum q-value for the given state.
        
        TODO: 
            1. return the action with maximum Q-value. Assume all relevant Q-values
               are stored in self.q[y, x]. The function np.argmax returns the index 
               of the maximum value (consult the numpy documentation online for 
               examples).
               
            2. implement an epsilon-greedy policy with random exploration probability 
               passed in through the parameter `epsilon`.
        
        """
        x, y = state
        q_s = self.q[y, x]
        a = 0 # set a to the correct return value
        return a
        
    
    def render_pi(self, env, ax=None):
        if ax is None:
            fig, ax = env._create_fig()
            
        env.render_grid(ax)
        for y in range(env.rows):
            for x in range(env.cols):
                env.render_action(ax, action=np.argmax(self.q[y, x]), agent=(x, y), length=0.3, hw=0.1, hl=0.15, ox=0, oy=0)

### 3.2 Q-Learning Update

Implement the Q-Learning update $Q(s_t, a_t) \leftarrow r + \gamma \, \text{argmax}_j \, Q(s_{t+1}, a_j)$

In [ ]:
# Q-Learning
env = ExampleGrid()
Q = np.zeros((env.rows, env.cols, len(env.action_space)))
agent = QAgent(q=Q)
epsilon = 1.0
gamma = 0.95

returns = []
for i in range(200):
    
    # do training rollout with epsilon-greedy policy
    states, actions, rewards = agent.rollout(env, epsilon)
    G = np.sum(rewards * gamma**np.arange(len(rewards)))
    returns.append(G)
    
    # update Q-Table
    for t in range(len(states)-1):
        x, y = states[t]
        a = actions[t]
        x_next, y_next = states[t+1]
        
        """
        TODO: implement the Q-Learning update.
        
        """
        Q[y, x, a] = Q[y, x, a] # an illustrative no-op that needs adapting.
    
    # decay epsilon
    epsilon = max(0.05, 0.9*epsilon)

# store returns for comparison
returns_q = returns

fig, ax = plt.subplots(3, 1, figsize=(10, 24))
ax[0].plot(returns_q)
ax[0].set_title('Q-Learning training return of each rollout')
ax[0].set_ylabel('Return')
ax[0].set_xlabel('Rollout')
agent.render_pi(env, ax[1])
ax[1].set_title('Q-Learning learned policy after training.')
env.render_rollout(states, ax=ax[2])
ax[2].set_title('Q-Learning example rollout after training.')

### 3.3 Evaluation

Evaluate your algorithm, training 5 times and plot mean, min and max return after each rollout

## 4. Does the task get harder to learn if the agent starts in a different random position?

Adapt the environmnet such that the the agent starts at a different (random) position in each rollout and explore the training progress of one RL algorithm. You may need to adapt the number of rollouts, the learning rate, or the exploration rate.

In [ ]:
class ExampleGrid(GridWorld):
    
    def __init__(self):
        """
        TODO: modify the assignment to `agent_start` below such that the agent starts 
              in a different random position (y, x) in each iteration.
        """
        super().__init__(rows=4, cols=5, agent_start=(0, 3))
        self.update_terminal(4, 0) # make state (4, 0) a terminal state
        self.update_reward(4, 0, 1) # make transitions into state (4, 0) yield a reward of 1
        self.update_reward(1, 2, -1) # make transitions into state (1, 2) yield a reward of -1
        self.agent_start = agent_start
        
env = ExampleGrid() # instantiate the example grid
env.render() # display the environment state(green dot), terminal states (gray shaded cells), and rewards (numbers inside cells)

In [ ]:
""" Evaluate one RL algorithm of you choice on the modified environment. """



## 5. Explore different tasks by modifying the environment's reward structure

Change the reward structure of the environment to represent a different custom task of your choice and explore the training progress of one RL algorithm, and interpret the learned policy. You may need to adapt the number of rollouts, the learning rate, or the exploration rate (epsilon).


In [ ]:
class ExampleGrid(GridWorld):
    
    def __init__(self):
        super().__init__(rows=4, cols=5, agent_start=(0, 3))
        self.update_terminal(4, 0) # make state (4, 0) a terminal state
        """
        TODO: change the reward structure to represent a different custom task.
        """
        self.update_reward(4, 0, 1) # make transitions into state (4, 0) yield a reward of 1
        self.update_reward(1, 2, -1) # make transitions into state (1, 2) yield a reward of -1
        
env = ExampleGrid() # instantiate the example grid
env.render() # display the environment state(green dot), terminal states (gray shaded cells), and rewards (numbers inside cells)

In [ ]:
""" Evaluate one RL algorithm of you choice on the modified environment. """
